In [2]:
#import all the required libraries
#import all the required libraries
import selenium
import pandas as pd
import os
from PIL import Image
import io
import requests
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import time
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
pd.set_option('display.max_rows', 120)
#wait = WebDriverWait(driver, 30)

### Question:1:- Write a python program which searches all the product under a particular product vertical from www.amazon.in. The product verticals to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [11]:
val = input("Enter Product To Be Searched: ")
driver=webdriver.Chrome("chromedriver.exe") 
time.sleep(3)
url = "https://www.amazon.in/ref=nav_logo"
driver.get(url)
time.sleep(2)
#locating the search bar
search_bar = driver.find_element_by_id("twotabsearchtextbox")    # Locating searc_bar by id
search_bar.clear()                                               # clearing search_bar
search_bar.send_keys(val)                                   # sending user input to search bar
search_button = driver.find_element_by_xpath('//span[@id="nav-search-submit-text"]')       # Locating search_button by xpath
search_button.click()


Enter Product To Be Searched: laptop



### Question 2:- In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a dataframe and csv. In case if any product vertical has less than 3 pages in search results then scrape all the products available under that product vertical. Details to be scraped are: "Brand Name", "Name of the Product", "Rating", "No. of Ratings", "Price", "Return/Exchange", "Expected Delivery", "Availability", "Other Details" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [12]:
brand = []
product_name = []
rating = []
count_rating = []
price = []
exchange = []
exp_delivery = []
availability = []
other_details = []
UR = []
for i in range(1,4):
    urls=driver.find_elements_by_xpath("//a[@class='a-link-normal a-text-normal']") #collecting urls of all product
    for ur in urls:
        try:
            UR.append(ur.get_attribute('href'))     #add product url in List
        except e:
            continue
    next_page = driver.find_element_by_xpath("//li[@class='a-last']") #Click on next page
    next_page.click()
    time.sleep(5)
    
for url in UR:              #access every url from UR list using for loop
    driver.get(url)               #access url
    try:                                     # Exception handling
        name = driver.find_element_by_xpath("//span[@class='a-size-large product-title-word-break']") # Get Product name from each URL
        product_name.append(name.text)
    except NoSuchElementException as e:
        product_name.append("-")
#        print(e)
        
    try:                                     # Exception handling for brand name.
        brand_tag = driver.find_element_by_xpath("//span[@class='a-size-large product-title-word-break']") # Get brand name from each URL
        brand.append(brand_tag.text.split(" ")[0])
    except NoSuchElementException as e:
        brand.append("-")
#        print(e)
        
    try:                                     # Exception handling for brand name.
        count_rating_tag = driver.find_element_by_id("acrCustomerReviewText")  # Get rating count for each product each URL
        count_rating.append(count_rating_tag.text)
    except NoSuchElementException as e:
        count_rating.append("-")
#        print(e) 
        
    try:                  #exception handling for nosuchelementexception                                                    #click the rating link found
        Rating=driver.find_element_by_xpath("//span[@class='a-size-base a-nowrap']//span") #locating the rating
        rating.append(Rating.text)     #appending the ratings in Ratings list
        
    except NoSuchElementException as e:
        rating.append("-")    #appending the No rating if no rating is there
        
    try:                  #exception handling for nosuchelementexception                                                    #click the rating link found
        Price=driver.find_element_by_id("priceblock_ourprice")  #locating the price
        price.append(Price.text)            #appending the price in price list
        
    except NoSuchElementException as e:
        price.append("-")           #appending the - if no price is there
    try:                             #exception handling for nosuchelementexception                                                    #click the rating link found
        avail=driver.find_element_by_xpath("//span[@class='a-size-medium a-color-success']")  #locating the item availability
        availability.append(avail.text)                            #appending the availability in list
        
    except NoSuchElementException as e:
        availability.append("-")                         #appending the - if no item avaialble 
        
    try:                  #exception handling for nosuchelementexception                                                    #click the rating link found
        exp_delivry=driver.find_element_by_xpath("//div[@class='a-section a-spacing-mini']/b")  #locating the expected delivery
        exp_delivery.append(exp_delivry.text)#appending the expected delivery in Ratings list
        
    except NoSuchElementException as e:
        exp_delivery.append("-")            #appending the - if delivery time given
                
    try:                  #exception handling for nosuchelementexception                                                    #click the rating link found
        exchange_tag=driver.find_element_by_xpath("//div[@class='a-row a-expander-container a-expander-extend-container']/div[3]")  #locating the rating
        x = exchange_tag.text.split(':')
        exchange.append(x[1])#              appending the exchange offer in list
        
    except NoSuchElementException as e:
        exchange.append("-")          #appending - if exchange offer details available
        
    try:                  #exception handling for nosuchelementexception                                                    #click the rating link found
        other_tag=driver.find_element_by_xpath("//ul[@class='a-unordered-list a-vertical a-spacing-mini']")  #locating other information
        other_details.append(other_tag.text)#appending the other informaiton in list
        
    except NoSuchElementException as e:
        other_details.append("-")           #appending - if other information no available

ds = pd.DataFrame({"Brand":brand,"Product_name":product_name,"Rating":rating,"Rating_Count":count_rating,"Price":price,"Exchange_offer":exchange,"Expected_Delivery":exp_delivery,"Availability":availability,"Other_info":other_details})

ds.head(10)

Brand                                       Product_name        Rating  \
0    ASUS  ASUS VivoBook 14 (2020) Intel Quad Core Pentiu...  3.7 out of 5   
1    ASUS  ASUS VivoBook 14 (2020) Intel Core i5-1035G1 1...  3.8 out of 5   
2    ASUS  ASUS ROG Zephyrus G14, 14" FHD 120Hz, Ryzen 5 ...  4.4 out of 5   
3  Lenovo  Lenovo Ideapad Slim 3 Intel Celeron N4020 15.6...             -   
4    ASUS  ASUS VivoBook 14 (2020) Intel Quad Core Pentiu...  3.7 out of 5   
5      HP  HP 14 (2021) Thin & Light Intel Celeron N4500 ...  3.5 out of 5   
6      HP  HP 15 Entry Level 15.6-inch (39.62 cms) HD Lap...  3.7 out of 5   
7      HP  HP 15 Intel Pentium Gold 6405U Processor Entry...  3.5 out of 5   
8      HP  HP 15 AMD Athlon 15.6" (39.62cms) HD Laptop (S...  3.9 out of 5   
9  Lenovo  Lenovo Ideapad S145 AMD RYZEN 3 3200U 15.6-inc...  3.3 out of 5   

  Rating_Count        Price                             Exchange_offer  \
0   84 ratings  ₹ 24,990.00   Exchange is unavailable for your pincode   
1   62 ratings  ₹ 39,990.00   Exchange is unavailable for your pincode   
2  124 ratings  ₹ 84,990.00   Exchange is unavailable for your pincode   
3            -  ₹ 25,615.00   Exchange is unavailable for your pincode   
4   84 ratings  ₹ 24,990.00   Exchange is unavailable for your pincode   
5   43 ratings  ₹ 29,490.00   Exchange is unavailable for your pincode   
6  702 ratings            -                                          -   
7  243 ratings  ₹ 27,594.00   Exchange is unavailable for your pincode   
8   25 ratings  ₹ 27,499.00   Exchange is unavailable for your pincode   
9  733 ratings  ₹ 28,990.00   Exchange is unavailable for your pincode   

   Expected_Delivery Availability  \
0       June 26 - 29            -   
1     Monday, May 31    In stock.   
2  Wednesday, June 2    In stock.   
3    Tuesday, June 1    In stock.   
4       June 26 - 29            -   
5    Tuesday, June 1    In stock.   
6                  -                
7     Monday, May 31    In stock.   
8     Monday, May 31    In stock.   
9     Monday, May 31    In stock.   

                                          Other_info  
0  Processor: 10th Gen Intel Pentium Silver N5030...  
1  Processor: 10th Gen Intel Core i5-1035G1, 1.0 ...  
2  Processor: AMD Ryzen 5 4600HS Processor, 3.0 G...  
3  Processor: Intel Celeron| Speed: 1.1 GHz (Base...  
4  Processor: 10th Gen Intel Pentium Silver N5030...  
5  Processor: Intel Celeron N4500 (1.1 GHz base c...  
6  Processor: AMD 3020e (Base Clock Speed 1.2 GHz...  
7  Design & Battery: Thin and light design | Lapt...  
8  Processor: AMD Athlon Silver 3050U (2.3 GHz ba...  
9  Processor: AMD Ryzen 3 3200U processor, 2.6 Gh...

## Question 3:- Write a python program to access the search bar and search button on images.google.com and scrape 100 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’.

In [8]:
## For this question we will define a function, which will take search key as argument and will scrap the images searched based
## Search key.

def img_scraping(word):
    #save base directory location
    baseDir=os.getcwd()
    driver=webdriver.Chrome("chromedriver.exe")
    url = "https://images.google.com/"
    driver.get(url)
    search = driver.find_element_by_xpath("//input[@class='gLFyf gsfi']")
    search.send_keys(word)
    button = driver.find_element_by_xpath("//span[@class='z1asCe MZy1Rb']")
    button.click()

    #Scroll to the end of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)#sleep_between_interactions
    
    #Scroll to the end of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)#sleep_between_interactions

    #Locate the images to be scraped from the current page 
    urls_tags = driver.find_elements_by_xpath("//a[contains(@class,'mM5pbd')]")
    totalResults=len(urls_tags)
    print(totalResults)
    time.sleep(10)
    #Click on each Image to extract its corresponding link to download

    img_urls = []
    for i in range(0,50):
        img=urls_tags[i]
        try:
            img.click()
            time.sleep(5)
            actual_images = driver.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'https' in actual_image.get_attribute('src'):
                    img_urls.append(actual_image.get_attribute('src'))
        except ElementClickInterceptedException or ElementNotInteractableException as err:
            print(err)

    #Download & save each image in the Destination directory
    #create directory in current location
    path = "img\\" + word
    os.mkdir(path)
    os.chdir(path)
    imgDir=os.getcwd()

    for i, url in enumerate(img_urls):
        file_name = f"{i:100}.jpg"    
        try:
            image_content = requests.get(url).content

        except Exception as e:
                print(f"ERROR - COULD NOT DOWNLOAD {url} - {e}")

        try:
            image_file = io.BytesIO(image_content)
            image = Image.open(image_file).convert('RGB')

            file_path = os.path.join(imgDir, file_name)

            with open(file_path, 'wb') as f:
                image.save(f, "JPEG", quality=85)
            print(f"SAVED - {url} - AT: {file_path}")
        except Exception as e:
            print(f"ERROR - COULD NOT SAVE {url} - {e}")
        #change dir to Base Directory
        os.chdir(baseDir) 
        
        
#img_scraping('Fruits')
#img_scraping('Cars')
img_scraping('Machine Learning')

200
SAVED - https://thumbor.forbes.com/thumbor/fit-in/1200x0/filters%3Aformat%28jpg%29/https%3A%2F%2Fspecials-images.forbesimg.com%2Fdam%2Fimageserve%2F1129869424%2F0x0.jpg%3Ffit%3Dscale - AT: C:\Users\hemkashi\Desktop\Science\Fliprobo\4.Webscarping-Project\img\Machine Learning\                                                                                                   0.jpg
SAVED - https://thumbor.forbes.com/thumbor/960x0/https%3A%2F%2Fspecials-images.forbesimg.com%2Fdam%2Fimageserve%2F966248982%2F960x0.jpg%3Ffit%3Dscale - AT: C:\Users\hemkashi\Desktop\Science\Fliprobo\4.Webscarping-Project\img\Machine Learning\                                                                                                   1.jpg
SAVED - https://www.expert.ai/wp-content/uploads/2017/03/machine-learning-definition.jpeg - AT: C:\Users\hemkashi\Desktop\Science\Fliprobo\4.Webscarping-Project\img\Machine Learning\                                                                                       

C:\Users\hemkashi\.conda\envs\machine-Learning\lib\site-packages\PIL\Image.py:962: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


SAVED - https://mobidev.biz/wp-content/uploads/2020/01/machine-learning-methods-demand-forecasting-1.png - AT: C:\Users\hemkashi\Desktop\Science\Fliprobo\4.Webscarping-Project\img\Machine Learning\                                                                                                  10.jpg
SAVED - https://www.securityroundtable.org/wp-content/uploads/2019/06/machine-learning.png - AT: C:\Users\hemkashi\Desktop\Science\Fliprobo\4.Webscarping-Project\img\Machine Learning\                                                                                                  11.jpg
SAVED - https://www.sas.com/en_in/insights/analytics/machine-learning/_jcr_content/par/styledcontainer_31e3/image.img.jpg/1614281913234.jpg - AT: C:\Users\hemkashi\Desktop\Science\Fliprobo\4.Webscarping-Project\img\Machine Learning\                                                                                                  12.jpg
SAVED - https://cdn-media-1.freecodecamp.org/images/1*92h6Lg1Bu1F9QqoVNrk

SAVED - https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcToqqMfOrfY6GZ4px8jFHkK_4NtQHwzHP3Npg&usqp=CAU - AT: C:\Users\hemkashi\Desktop\Science\Fliprobo\4.Webscarping-Project\img\Machine Learning\                                                                                                  38.jpg
SAVED - https://www.ciat.org/wp-content/uploads/2020/06/2020_blog_villalon_toolbox-1.png - AT: C:\Users\hemkashi\Desktop\Science\Fliprobo\4.Webscarping-Project\img\Machine Learning\                                                                                                  39.jpg
SAVED - https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSEMHTPfP9YZM7V99dKJrGBiDwui2iBsniBjg&usqp=CAU - AT: C:\Users\hemkashi\Desktop\Science\Fliprobo\4.Webscarping-Project\img\Machine Learning\                                                                                                  40.jpg
SAVED - https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSFueOb66mkdjk_FrOsnYVpzw7U3AwkhUOB5g&usqp=

SAVED - https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTQ0amx-Bk0awsXuwmaUdtFQPZFknvsQ0PRgA&usqp=CAU - AT: C:\Users\hemkashi\Desktop\Science\Fliprobo\4.Webscarping-Project\img\Machine Learning\                                                                                                  65.jpg
SAVED - https://miro.medium.com/max/848/1*M9le42saJxWlOYyYvhKtPA.jpeg - AT: C:\Users\hemkashi\Desktop\Science\Fliprobo\4.Webscarping-Project\img\Machine Learning\                                                                                                  66.jpg
SAVED - https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQPRh_bHRkCuP3PSyEYC_gw8mrKONs57eeBXw&usqp=CAU - AT: C:\Users\hemkashi\Desktop\Science\Fliprobo\4.Webscarping-Project\img\Machine Learning\                                                                                                  67.jpg
SAVED - https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSMFWgErcxBOju7NYcTXjCkOAcL_D7xVuskdw&usqp=CAU - AT: C:\Users\

ERROR - COULD NOT SAVE https://www.techfunnel.com/wp-content/uploads/2020/12/machine-learning-vs-deep-learning.png - cannot identify image file <_io.BytesIO object at 0x000001356DC40630>
SAVED - https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRqcK2lsBZ7SMqDG4cL0JS9AT8qAROcNDfMQA&usqp=CAU - AT: C:\Users\hemkashi\Desktop\Science\Fliprobo\4.Webscarping-Project\img\Machine Learning\                                                                                                  94.jpg
SAVED - https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcThKWzCEIz4Qlh_hyRMcifUF8AUU-dK_N7iRg&usqp=CAU - AT: C:\Users\hemkashi\Desktop\Science\Fliprobo\4.Webscarping-Project\img\Machine Learning\                                                                                                  95.jpg
SAVED - https://data-flair.training/blogs/wp-content/uploads/sites/2/2017/07/applications-of-machine-learning-2-1200x720.jpg - AT: C:\Users\hemkashi\Desktop\Science\Fliprobo\4.Webscarping-Project\img\Mac

## Question 4:- Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Display Resolution”, “Processor”, “Processor Cores”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [17]:
driver=webdriver.Chrome("chromedriver.exe") 
wait = WebDriverWait(driver, 5)
time.sleep(3)
url = "https://www.flipkart.com/"
driver.get(url)
time.sleep(10)
#Cancle Login Window
wait.until(EC.element_to_be_clickable((By.XPATH, "//button[@class='_2KpZ6l _2doB4z']"))).click()   #Cancel login button
#locating the search bar
search_bar=driver.find_element_by_class_name("_3704LK")
search_bar.send_keys('pixel 4A')

#locating the button and clicking it toh search for sunglasses
button=driver.find_element_by_class_name('L0Z3Pu')
button.click()
time.sleep(5)
urls=driver.find_elements_by_xpath("//a[@class='_1fQZEK']") #collecting urls of all product

UR = []
for ur in urls:
    UR.append(ur.get_attribute('href'))
Brand_Name =[]
Smartphone_name =[]
Colour =[]
RAM=[]
Storage=[]
Primary_Camera =[]
Secondary_Camera=[]
Display_Size = []
Display_Resolution =[]
Processor = []
Processor_Cores=[]
Battery_Capacity=[]
Price = []

for url in UR:
    driver.get(url)
    time.sleep(5)
    driver.find_element_by_xpath("//button[@class='_2KpZ6l _1FH0tX']").click() # click to display all information
    Brand_Name.append(driver.find_element_by_xpath("//span[@class='B_NuCI']").text.split(" ")[0])
#    Smartphone_name.append(driver.find_element_by_xpath("//div[@class='_1UhVsV']/div[1]/table/tbody/tr[3]/td[2]").text)
    Smartphone_name.append(driver.find_element_by_xpath("//span[@class='B_NuCI']").text)
    Price.append(driver.find_element_by_xpath("//div[@class='_30jeq3 _16Jk6d']").text)
    Colour.append(driver.find_element_by_xpath("//div[@class='_1UhVsV']/div[1]/table/tbody/tr[4]/td[2]").text)
    Display_Size.append(driver.find_element_by_xpath("//div[@class='_1UhVsV']/div[2]/table/tbody/tr[1]/td[2]").text)
    Display_Resolution.append(driver.find_element_by_xpath("//div[@class='_1UhVsV']/div[2]/table/tbody/tr[2]/td[2]").text)
    Processor.append(driver.find_element_by_xpath("//div[@class='_1UhVsV']/div[3]/table/tbody/tr[2]/td[2]").text)
    Processor_Cores.append(driver.find_element_by_xpath("//div[@class='_1UhVsV']/div[3]/table/tbody/tr[3]/td[2]").text)
    Storage.append(driver.find_element_by_xpath("//div[@class='_1UhVsV']/div[4]/table/tbody/tr[1]/td[2]").text)
    RAM.append(driver.find_element_by_xpath("//div[@class='_1UhVsV']/div[4]/table/tbody/tr[2]/td[2]").text)
    try:
        Primary_Camera.append(driver.find_element_by_xpath("//div[@class='_1UhVsV']/div[5]/table/tbody/tr[2]/td[2]").text)
    except NoSuchElementException:
        Primary_Camera.append("-")
    try:
        Secondary_Camera.append(driver.find_element_by_xpath("//div[@class='_1UhVsV']/div[5]/table/tbody/tr[5]/td[2]").text)
    except NoSuchElementException:
        Secondary_Camera.append("-")
    Battery_Capacity.append(driver.find_element_by_xpath("//div[@class='_1UhVsV']/div[9]/table/tbody/tr[1]/td[2]").text)


ds = pd.DataFrame({"Brand":Brand_Name,
                   "Name":Smartphone_name,
                   "Color":Colour,
                   "RAM":RAM,
                   "Storage":Storage,
                   "Primary_Camera":Primary_Camera,
                   "Secondary_Camera":Secondary_Camera,
                   "Display_Size":Display_Size,
                   "Display_Resolution":Display_Resolution,
                   "Processor":Processor,
                   "Core":Processor_Cores,
                   "Battery":Battery_Capacity,
                   "Price":Price })

ds.to_csv("Phone_data.csv")
ds

Brand                                               Name  \
0   Google   Google Pixel 4a (Just Black, 128 GB)  (6 GB RAM)   
1    REDMI  REDMI 9 Power (Electric Green, 64 GB)  (4 GB RAM)   
2    REDMI      REDMI Note 9 (Pebble Grey, 64 GB)  (4 GB RAM)   
3    REDMI    REDMI 9 Power (Blazing Blue, 64 GB)  (4 GB RAM)   
4    REDMI    REDMI 9 Power (Mighty Black, 64 GB)  (4 GB RAM)   
5    REDMI       REDMI 9 Power (Fiery Red, 64 GB)  (4 GB RAM)   
6    REDMI     REDMI Note 9 (Shadow Black, 64 GB)  (4 GB RAM)   
7    REDMI       REDMI Note 9 (Aqua Green, 64 GB)  (4 GB RAM)   
8    Redmi              Redmi 9 (Sky Blue, 64 GB)  (4 GB RAM)   
9    Redmi          Redmi 9 (Carbon Black, 64 GB)  (4 GB RAM)   
10   REDMI     REDMI Note 9 (Arctic White, 64 GB)  (4 GB RAM)   
11   REDMI   REDMI 9 Power (Mighty Black, 128 GB)  (6 GB RAM)   
12   REDMI   REDMI 9 Power (Mighty Black, 128 GB)  (4 GB RAM)   
13   Redmi         Redmi 9 (Sporty Orange, 64 GB)  (4 GB RAM)   
14   REDMI     REDMI 9 Prime (Matte Black, 64 GB)  (4 GB RAM)   
15   REDMI   REDMI 9 Power (Blazing Blue, 128 GB)  (6 GB RAM)   
16   Redmi         Redmi 9A (Nature Green, 32 GB)  (2 GB RAM)   
17   Redmi       Redmi 9A (Midnight Black, 32 GB)  (2 GB RAM)   
18   Redmi              Redmi 9A (SeaBlue, 32 GB)  (2 GB RAM)   
19    ViVO        ViVO Y20G (Purist Blue, 128 GB)  (6 GB RAM)   
20    Itel  Itel VISION1 (GRADATION GREEN, 32 GB)  (2 GB RAM)   
21   REDMI     REDMI Note 9 (Pebble Grey, 128 GB)  (4 GB RAM)   
22   REDMI  REDMI 9 Power (Electric Green, 128 GB)  (6 GB ...   
23   REDMI    REDMI Note 9 (Arctic White, 128 GB)  (4 GB RAM)   

              Color   RAM Storage      Primary_Camera       Secondary_Camera  \
0        Just Black  6 GB  128 GB  12.2MP Rear Camera       8MP Front Camera   
1    Electric Green  4 GB   64 GB    48MP Rear Camera       8MP Front Camera   
2       Pebble Grey  4 GB   64 GB    48MP Rear Camera  13MP AI Selfie Camera   
3      Blazing Blue  4 GB   64 GB    48MP Rear Camera       8MP Front Camera   
4      Mighty Black  4 GB   64 GB    48MP Rear Camera       8MP Front Camera   
5         Fiery Red  4 GB   64 GB    48MP Rear Camera       8MP Front Camera   
6      Shadow Black  4 GB   64 GB    48MP Rear Camera  13MP AI Selfie Camera   
7        Aqua Green  4 GB   64 GB    48MP Rear Camera  13MP AI Selfie Camera   
8          Sky Blue  4 GB   64 GB          13MP + 2MP       5MP Front Camera   
9      Carbon Black  4 GB   64 GB          13MP + 2MP       5MP Front Camera   
10     Arctic White  4 GB   64 GB    48MP Rear Camera  13MP AI Selfie Camera   
11     Mighty Black  6 GB  128 GB    48MP Rear Camera       8MP Front Camera   
12     Mighty Black  4 GB  128 GB    48MP Rear Camera       8MP Front Camera   
13    Sporty Orange  4 GB   64 GB          13MP + 8MP       5MP Front Camera   
14      Matte Black  4 GB   64 GB    13MP Rear Camera       8MP Front Camera   
15     Blazing Blue  6 GB  128 GB    48MP Rear Camera       8MP Front Camera   
16     Nature Green  2 GB   32 GB    13MP Rear Camera                    Yes   
17   Midnight Black  2 GB   32 GB    13MP Rear Camera                    Yes   
18          SeaBlue  2 GB   32 GB                   -                      -   
19      Purist Blue  6 GB  128 GB      Primary Camera                      -   
20  GRADATION GREEN  2 GB   32 GB     8MP Rear Camera                    Yes   
21      Pebble Grey  4 GB  128 GB    48MP Rear Camera  13MP AI Selfie Camera   
22   Electric Green  6 GB  128 GB    48MP Rear Camera       8MP Front Camera   
23     Arctic White  4 GB  128 GB    48MP Rear Camera  13MP AI Selfie Camera   

             Display_Size  Display_Resolution                  Processor  \
0    14.76 cm (5.81 inch)  2340 x 1080 Pixels   Qualcomm Snapdragon 730G   
1    16.59 cm (6.53 inch)  2340 x 1080$$pixel    Qualcomm Snapdragon 662   
2    16.59 cm (6.53 inch)  2340 x 1080$$pixel         MediaTek Helio G85   
3    16.59 cm (6.53 inch)  2340 x 1080$$pixel    Qualcomm Snapdragon 662  

## Question 5:- Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [56]:
val = input("Enter Product To Be Searched: ")
#val = 'Delhi'
driver=webdriver.Chrome("chromedriver.exe") 
wait = WebDriverWait(driver, 5)
time.sleep(3)
url = "https://www.google.co.in/maps"
driver.get(url)
time.sleep(10)
search = driver.find_element_by_xpath("//input[@class='tactile-searchbox-input']")
search.send_keys(val)
driver.find_element_by_xpath("//button[@class='searchbox-searchbutton']").click()
time.sleep(5)
url = driver.current_url
latitude = url.split("/")[6].split(",")[0].strip("@")
longitude = url.split("/")[6].split(",")[1]

print("Latitude of %s is %s"%(val,latitude))
print("Longitude of %s is %s"%(val,longitude))

Enter Product To Be Searched: Kota
Latitude of Kota is 25.1733863
Longitude of Kota is 75.7769074


## Question 6:- Write a program to scrap details of all the funding deals for second quarter (i.e. July 20 – September 20) from trak.in.

In [5]:
#Load module required to handle drop down manu

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

pd.set_option('display.max_rows', None) ##to display all the Rows
driver=webdriver.Chrome("chromedriver.exe")  #Load Chrome Web driver
time.sleep(3)
url = "https://trak.in"  # Url 
driver.get(url)   #Browse URL
driver.maximize_window()   ## Maximize window, so that fund detail tab is visible.

l=driver.find_element_by_xpath("//a[@title='https://trak.in/india-startup-funding-investment-2015/']")   # Access Funding details
l.click()            # Click on Funding Detail tab
 
time.sleep(10)           # Sleep for 10 second to allow browser to load the url properly 

select = Select(driver.find_element_by_name("tablepress-49_length"))    # access table row display drop down to display all 12 records
select.select_by_value("25")                                              #Select 25 row fom drop down.


select = Select(driver.find_element_by_name("tablepress-50_length"))       # access table row display drop down to display all 12 records
select.select_by_value("25")                                               #Select 25 row fom drop down.

time.sleep(2)                                                                 
header = driver.find_elements_by_xpath("//div[@id='tablepress-48_wrapper']/div[3]/div/div/table/thead/tr/th")  # Access table Header


Header = []                                         # Defind empty header list
for i in header:                                     #store all header from web element to list using for loop
    Header.append(i.text)
    
df = pd.DataFrame(columns = Header)                #create a empty dataframe having header definition only.
col = len(Header)                                   #get leanth of Header List , it will be used during data processing.

# Access table data for first page.
page1_jul = driver.find_elements_by_xpath("//div[@id='tablepress-48_wrapper']/div[3]/div[2]/table/tbody/tr")   #Access rows for july fund table
page1_aug = driver.find_elements_by_xpath("//div[@id='tablepress-49_wrapper']/div[3]/div[2]/table/tbody/tr")   #Access rows for Aug Fund table
page1_sep = driver.find_elements_by_xpath("//div[@id='tablepress-50_wrapper']/div[3]/div[2]/table/tbody/tr")   # Access rows for Sep Fund table
quarter = [page1_jul,page1_aug,page1_sep]                           # create list of all the web element we have collection for each month.

for month in quarter:                                 # loop on each quarter list
    data = {}                                         #create a empty  data disctionry 
    for p in range(1,col+1):                          # inner loop for each column
        str1,str2 = "td[" ,"]"                        #create two string varible
        path = str1+str(p)+str2                        #create a dynamic xpath which contain counter value in xpath 
        temp = []                                      #temporary list for data processing
        for i in month:                                # Process for each mont data.
            temp.append(i.find_element_by_xpath(path).text)   #append data in temp list
        data.update({p:temp})                            #append temp list in data disctionry column wise
    df_temp = pd.DataFrame(data)                         # for each quarter create a temporary data frame
    df_temp.columns= Header                             # Set header in temp data frame
    df = pd.concat([df,df_temp], ignore_index=True,axis=0)  #Finally marg ps data frame and temp data frame and loop again for each quarter 


df                      #Disaply content of data Frame.

Sr. No. Date (dd/mm/yyyy)               Startup Name  \
0        1        15/07/2020                   Flipkart   
1        2        16/07/2020                    Vedantu   
2        3        16/07/2020                       Crio   
3        4        14/07/2020                    goDutch   
4        5        13/07/2020                   Mystifly   
5        6        09/07/2020               JetSynthesys   
6        7        10/07/2020                   gigIndia   
7        8        15/07/2020                  PumPumPum   
8        9        14/07/2020                       FLYX   
9       10        13/07/2020  Open Appliances Pvt. Ltd.   
10       1        15/08/2020                     Practo   
11       2        13/08/2020                    Medlife   
12       3        13/08/2020                  HungerBox   
13       4        04/08/2020                      Dunzo   
14       5        11/08/2020                   Terra.do   
15       6        12/08/2020                  Classplus   
16       7        14/08/2020                       Niyo   
17       8        10/08/2020                  ZestMoney   
18       9        07/08/2020                FreshToHome   
19      10        13/08/2020                    Eduvanz   
20      11        03/08/2020                 CrowdPouch   
21      12        04/08/2020                 DrinkPrime   
22       1        08/09/2020                     Byju’s   
23       2        12/09/2020                  mCaffeine   
24       3        09/09/2020                     Qshala   
25       4        02/09/2020                      Winzo   
26       5        09/09/2020                Hippo Video   
27       6        07/09/2020                    Melorra   
28       7        07/09/2020                        1mg   
29       8        31/08/2020                      mfine   
30       9        31/08/2020                       Apna   
31      10        03/09/2020                    Railofy   
32      11        08/09/2020            Cell Propulsion   

                       Industry / Vertical  \
0                               E-commerce   
1                                  EduTech   
2                                  EduTech   
3                                  FinTech   
4                      Airfare Marketplace   
5                 Gaming and Entertainment   
6                              Marketplace   
7                        Automotive Rental   
8                               OTT Player   
9                   Information Technology   
10                              HealthTech   
11                              E-commerce   
12                                FoodTech   
13                   Hyper-local Logistics   
14                                 EduTech   
15                                 EduTech   
16                                 FinTech   
17                                 FinTech   
18                              E-commerce   
19                                 FinTech   
20                                 FinTech   
21                      Water Purification   
22                                 EduTech   
23                           Personal Care   
24                                 EduTech   
25                           Online Gaming   
26  Video Customer Experience(CX) Platform   
27                              E-commerce   
28                              E-commerce   
29                              HealthTech   
30                         Human Resources   
31                          Transportation   
32                              Automobile   

                                         Sub-Vertical  \
0                                          E-commerce   
1                                     Online Tutoring   
2                    Learning Platform for Developers   
3                                      Group Payments   
4   Ticketing, Airline Retailing, and Post-Ticketi...   
5                            Gaming and Entertainment   
6                           Crow

## Question 7:- Write a program to scrap all the available details of top 10 gaming laptops from digit.in.

In [3]:
driver=webdriver.Chrome("chromedriver.exe") 
time.sleep(2)
url = "https://www.digit.in/top-products/best-gaming-laptops-40.html"   # get url of top gaming laptops
driver.get(url)
urls = driver.find_elements_by_xpath("//span[@class='datahreflink spec']")
UR = []
for ur in urls:   # get all laptops url from the list.
    UR.append(ur.get_attribute('data-href'))
basic_details = []

for url in UR:   # browse every URL using for loop
    driver.get(url)   #access every url for each laptop
    basic = {}        #defind a empty dictionry 
    basic['name'] = driver.find_element_by_xpath("//div[@class='heading-wraper']/h1").text  # get name of Laptop
    basic_info = driver.find_element_by_id("specs-basic-information")      #get table basic information
    rows = basic_info.find_elements_by_xpath("div[2]/table/tbody/tr")      #get all the row in basic info table
    for row in rows:                                                    #Process every row.
        x = row.text.split(":")[0]                                    # collect Key of dictionary 
        y = row.text.split(":")[1]                                    #Value of Dictionary
        basic[x] = y                                                  #Add key value in dictionary 
    dis_info = driver.find_element_by_id("specs-display")    # get table display info
    rows = dis_info.find_elements_by_xpath("div[2]/table/tbody/tr")           #get all the row in basic info table
    for row in rows:                                                          #Process every row.
        x = row.text.split(":")[0]                                            # collect Key of dictionary
        y = row.text.split(":")[1]                                            #Value of Dictionary
        basic[x] = y                                                          #Add key value in dictionary
    
    memory_info = driver.find_element_by_id("specs-memory")    # access memory info 
    rows = memory_info.find_elements_by_xpath("div[2]/table/tbody/tr")        #get all the row in memory info table
    for row in rows:                                                          #run loop for every loop and extract key value
        x = row.text.split(":")[0]                                           #from each row and add them in basic dictionary
        y = row.text.split(":")[1]
        basic[x] = y

    processor_info = driver.find_element_by_id("specs-processor")            #access processo info
    rows = processor_info.find_elements_by_xpath("div[2]/table/tbody/tr")       
    for row in rows:        
        x = row.text.split(":")[0]         
        y = row.text.split(":")[1]
        basic[x] = y                                                      #store in dictionary in key value pair
    
    storage_info = driver.find_element_by_id("specs-storage")             #get storage information
    rows = storage_info.find_elements_by_xpath("div[2]/table/tbody/tr")       
    for row in rows:        
        x = row.text.split(":")[0]
        y = row.text.split(":")[1]
        basic[x] = y                                                #store in dictionary
    
    try:
        power_info = driver.find_element_by_id("specs-power")                        #get battery information
        rows = power_info.find_elements_by_xpath("div[2]/table/tbody/tr")       
        for row in rows:        
            x = row.text.split(":")[0]
            y = row.text.split(":")[1]
            basic[x] = y                                                      
    except NoSuchElementException:
        pass
    
    try:
        sound_info = driver.find_element_by_id("specs-power")                     #get sound information
        rows = sound_info.find_elements_by_xpath("div[2]/table/tbody/tr")       
        for row in rows:        
            x = row.text.split(":")[0]
            y = row.text.split(":")[1]
            basic[x] = y 
    except NoSuchElementException:
        pass
    
    try:
        warranty_info = driver.find_element_by_id("specs-warranty-and-manufacturer-info")      #get warrenty information
        rows = warranty_info.find_elements_by_xpath("div[2]/table/tbody/tr")       
        for row in rows:        
            x = row.text.split(":")[0]
            y = row.text.split(":")[1]
            basic[x] = y
    except NoSuchElementException:
        pass
        
    try:
        basic['Digital Rating'] = driver.find_element_by_xpath("//div[@class='Numeric-Number yellowtext']").text   #collect Digital Rating
    except NoSuchElementException:
        pass
    
    try:
        basic['Pros Cons'] = driver.find_element_by_xpath("//div[@class='pros-Cons']").text.replace('\n',' ')   #Pros and Cons of laptop
    except NoSuchElementException:
        pass
        
    basic_details.append(basic)                    #append dictionary in list
    
ds = pd.DataFrame(basic_details)

ds

name           Model Name   \
0     Alienware Area 51M R2           Area 51M R2   
1          Alienware M15 R3                m15 R3   
2    ASUS ROG Strix Scar 15                  ASUS   
3     Asus ROG Zephyrus G14      ROG Zephyrus G14   
4          Lenovo Legion 5i             Legion 5i   
5  Asus ROG Zephyrus Duo 15   ROG Zephyrus Duo 15   
6      Acer Aspire 7 Gaming       Aspire 7 Gaming   

  Operating System (With Version)    Resolution  Display Size (In Inches)   \
0                  Windows 10 Home   1920 x 1080                      17.3   
1                  Windows 10 Home   3840 x 2160                      15.6   
2                  Windows 10 Home   1920 x 1080                      15.6   
3                  Windows 10 Home   1920 x 1080                        14   
4                   Windows 10 Pro   1920 x 1080                      15.6   
5                       Windows 10   3840 x 1100                      15.6   
6                  Windows 10 Home   1920 x 1080                      15.6   

  Ram Included (In Gb)  Ram Type                      Processor Model Name   \
0                 16 GB      DDR4     10th Generation Intel® Core™ i7-10700   
1                 16 GB      DDR4   10th Generation Intel® Core™ i9-10980HK   
2                 16 GB      DDR4                       AMD Ryzen™ 9 5900HX   
3                 16 GB      DDR4                AMD 3rd Generation Ryzen 9   
4                 16 GB      DDR4    10th Generation Intel® Core™ i5-10300H   
5                  4 GB      DDR4             Intel Core i7 10th Gen 10875H   
6                  8 GB      DDR4              AMD Ryzen™ 5-5500U hexa-core   

  Clock Speed       Cores   ... Storage Drive Type  Storage Drive Capacity   \
0     2.90 GHz           8  ...                 SSD                    1 TB   
1          NaN           8  ...                 SSD                    1 TB   
2      3.3 GHz         NaN  ...                 SSD                    1 TB   
3      3.3 GHz   Quad Core  ...                 SSD                    1 TB   
4     2.50 GHz           4  ...                 SSD                    1 TB   
5          NaN         NaN  ...                 SSD                  512 GB   
6          NaN         NaN  ...                 SSD                  512 GB   

  Battery Type  Power Supply  Digital Rating  \
0           NaN           NaN            NaN   
1           NaN           NaN            NaN   
2        Li-ion          240W             73   
3       Lithium          180W             71   
4           NaN           NaN            NaN   
5           NaN           NaN            NaN   
6        Li-ion         135 W             75   

                                           Pros Cons Laptop Type   \
0                                                NaN          NaN   
1                                                NaN          NaN   
2  Pros + 1440p, 165Hz display is great for gamin...          NaN   
3  Pros + Impressive performance for the form-fac...       Gaming   
4                                                NaN          NaN   
5                                                NaN          NaN   
6  Pros + Understated design + Great performance ...          NaN   

  Battery Backup (In Hours)  Refresh Rate   \
0                        NaN           NaN   
1                        NaN           NaN   
2                        NaN           NaN   
3                    4 Hours           NaN   
4                  7.7 hours           NaN   
5                        NaN         60 Hz   
6                 11.50 Hour           NaN   

                                   Warranty Length   
0                                               NaN  
1                                               NaN  
2                                               NaN  
3                                               NaN  
4                                               NaN  
5                                               NaN  
6   One-year Internati